In [ ]:
import os
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import logging

# --- 프로젝트 루트 경로 설정 --- 
try:
    # 이 스크립트 파일(notebooks/...)의 상위 폴더(notebooks)의 상위 폴더(프로젝트 루트)
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    # __file__ 변수가 없는 환경 (e.g., REPL)을 위한 대체
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to path: {PROJECT_ROOT}")

# --- src 모듈 임포트 --- 
try:
    from src.utils import load_config, load_jsonl
    # Tmax 실험용 평가 함수 (src/evaluation.py 에 있다고 가정)
    from src.evaluation import calculate_qa_metrics_for_item 
    # from factscore import FactScorer # LongWiki 평가 시 필요 (예시)
except ImportError as e:
    print(f"Error importing src modules: {e}")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
logger = logging.getLogger(__name__)

print("Libraries and modules imported successfully.")

# Matplotlib/Seaborn 기본 스타일 설정
sns.set_theme(style="whitegrid", context="talk")

In [ ]:
# 설정 파일 로드 (결과 디렉토리 경로 확인용)
CONFIG_PATH = os.path.join(PROJECT_ROOT, "configs", "config.yaml")
config = load_config(CONFIG_PATH)

# Tmax 예비 실험 결과가 저장된 디렉토리 경로
# config.yaml에 'default_tmax_output_dir' 키로 정의했거나, 'run_tmax_experiment.py'의 기본값 사용
TUNING_RESULTS_DIR = os.path.join(PROJECT_ROOT, config.get('default_tmax_output_dir', 'results/hyperparam_tuning'))
print(f"Loading results from: {TUNING_RESULTS_DIR}")

# 파일 이름에서 파라미터 추출을 위한 정규 표현식
# 예: serc_t3_mf5_...jsonl -> t=3, mf=5
param_regex = re.compile(r"serc_t(\d+)_mf(\d+)_.*\\.jsonl")

all_results_flat = []
file_paths = glob.glob(os.path.join(TUNING_RESULTS_DIR, "**", "*.jsonl"), recursive=True)

if not file_paths:
    logger.warning(f"[경고] 예비 실험 결과 파일을 찾을 수 없습니다: {TUNING_RESULTS_DIR}")
    logger.warning("experiments/run_tmax_experiment.py를 먼저 실행하세요.")
else:
    print(f"Found {len(file_paths)} result files.")

for f_path in file_paths:
    filename = os.path.basename(f_path)
    match = param_regex.search(filename)
    
    if not match:
        logger.warning(f"Skipping file (name format mismatch): {filename}")
        continue
        
    # 파일명에서 T_max, Max_Facts 파싱
    t_max_param = int(match.group(1))
    max_facts_param = int(match.group(2))
    
    # 모델명, 데이터셋명 파싱 (경로 구조 기반)
    try:
        parts = f_path.replace("\\", "/").split('/')
        dataset_name = parts[-2]
        model_name_raw = parts[-3]
        model_name = model_name_raw.replace('_', '/') # e.g., meta-llama_Llama... -> meta-llama/Llama...
    except IndexError:
        model_name = "unknown"
        dataset_name = "unknown"

    # 결과 파일 로드
    results_data = load_jsonl(f_path)
    
    for item in results_data:
        # Tmax 실험 스크립트가 저장한 cycle_results (중간 결과) 추출
        # (run_tmax_experiment.py의 run_single_item_for_tmax 참조)
        cycle_results = item.get("tmax_eval_result", {}).get("cycle_results", [])
        
        for cycle_data in cycle_results:
            cycle_num = cycle_data.get('cycle') # 0 (baseline), 1, 2, ...
            
            # t_max_param (파일명의 T_max)보다 큰 사이클 결과는 무시 (일관성)
            if cycle_num > t_max_param:
                continue
                
            # 분석에 필요한 데이터만 평면화하여 추가
            flat_entry = {
                'model': model_name,
                'dataset': dataset_name,
                't_max_setting': t_max_param, # 이 실험의 T_max 설정값
                'max_facts_setting': max_facts_param, # 이 실험의 Max_Facts 설정값
                'cycle': cycle_num, # 실제 사이클 번호 (0~T_max)
                'query': item.get('query', item.get('question')),
                'item_id': item.get('id', item.get('hallulens_id', item.get('multispanqa_id', None))), # 고유 ID
                
                # 평가 지표 (Tmax 스크립트가 저장한 값 사용)
                # (이 예시는 PreciseWikiQA/MultiSpanQA의 EM/F1 기준)
                'f1': cycle_data.get('f1', 0.0), # calculate_qa_metrics_for_item 결과
                'em': cycle_data.get('em', 0.0), # calculate_qa_metrics_for_item 결과
                # 'factscore': cycle_data.get('factscore', 0.0), # LongWiki 사용 시
                # 'tokens': cycle_data.get('cumulative_tokens', 0) # Tmax 스크립트에서 토큰 수 저장이 구현되었다면
            }
            all_results_flat.append(flat_entry)

# 리스트를 Pandas DataFrame으로 변환
df_results = pd.DataFrame(all_results_flat)

if df_results.empty:
    logger.error("처리할 예비 실험 결과가 없습니다. 스크립트를 종료합니다.")
else:
    print("\n--- 데이터프레임 변환 완료 ---")
    display(df_results.head())
    print("\n--- 데이터프레임 정보 ---")
    df_results.info()

In [ ]:
if not df_results.empty:
    # 분석의 일관성을 위해, max_facts 값을 하나 정해서 T_max 효과 확인
    # (또는 모든 max_facts 설정에 대해 평균낼 수 있음)
    # 여기서는 예시로 가장 작은 max_facts 값 기준으로 필터링
    target_max_facts = df_results['max_facts_setting'].min()
    print(f"\nT_max 분석을 위해 max_facts_setting = {target_max_facts} 기준으로 필터링합니다.")
    
    df_tmax_analysis = df_results[
        (df_results['max_facts_setting'] == target_max_facts)
    ].copy()

    # 사이클(cycle)별 평균 성능 집계
    # (평가 지표는 f1 또는 factscore 등 데이터셋에 맞게 선택)
    primary_metric = 'f1' if 'f1' in df_tmax_analysis.columns else 'factscore' # 자동 선택 (임시)
    
    df_tmax_agg = df_tmax_analysis.groupby(['model', 'dataset', 'cycle']).agg(
        metric_avg=(primary_metric, 'mean')
        # tokens_avg=('tokens', 'mean')
    ).reset_index()
    
    print("\n--- T_max (Cycle) 별 평균 성능 ---")
    display(df_tmax_agg)
    
    # 시각화: T_max (Cycle)에 따른 성능 점수 변화
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=df_tmax_agg,
        x='cycle',
        y='metric_avg',
        hue='model', # 모델별로 다른 색상
        style='dataset', # 데이터셋별로 다른 스타일
        marker='o', 
        markersize=10
    )
    plt.title(f"T_max (Cycle)에 따른 {primary_metric.upper()} Score 변화 (Max_Facts={target_max_facts})")
    plt.xlabel("SERC Iteration Cycle (t) (0 = Baseline)")
    plt.ylabel(f"Average {primary_metric.upper()} Score")
    if not df_tmax_agg.empty:
        plt.xticks(df_tmax_agg['cycle'].unique()) # 0, 1, 2, ...
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    
    # (필요시 토큰 사용량 그래프도 추가)

In [ ]:
if not df_results.empty:
    # T_max=1 (1-pass) 일 때의 max_facts 효과 분석
    # (또는 예비 실험에서 결정된 최적 T_max=T_opt일 때의 결과로 분석)
    target_t_max = 1 # 예시로 1-pass 기준
    print(f"\nMax_Facts 분석을 위해 T_max = {target_t_max} 기준으로 필터링합니다.")

    df_maxfacts_analysis = df_results[
        (df_results['cycle'] == target_t_max) # T=1 사이클의 결과만
    ].copy()

    # Max_Facts 설정값별 평균 성능 집계
    primary_metric = 'f1' if 'f1' in df_results.columns else 'factscore' # 자동 선택 (임시)

    df_maxfacts_agg = df_maxfacts_analysis.groupby(['model', 'dataset', 'max_facts_setting']).agg(
        metric_avg=(primary_metric, 'mean')
        # tokens_avg=('tokens', 'mean') # T=1일 때 토큰 수 비교
    ).reset_index()

    print(f"\n--- MAX_FACTS_PER_GROUP 별 평균 성능 (T_max={target_t_max} 기준) ---")
    display(df_maxfacts_agg)

    # 시각화: Max_Facts에 따른 성능 점수 변화
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=df_maxfacts_agg,
        x='max_facts_setting',
        y='metric_avg',
        hue='model', # 모델별
        style='dataset', # 데이터셋별
        marker='o', 
        markersize=10
    )
    plt.title(f"MAX_FACTS_PER_GROUP에 따른 {primary_metric.upper()} Score 변화 (T_max={target_t_max} 기준)")
    plt.xlabel("MAX_FACTS_PER_GROUP (max_facts_setting)")
    plt.ylabel(f"Average {primary_metric.upper()} Score (at T={target_t_max})")
    if not df_maxfacts_agg.empty:
        plt.xticks(df_maxfacts_agg['max_facts_setting'].unique())
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    
    # (필요시 토큰 사용량 그래프도 추가)